## XFOIL analysis with python UI

    Commend "LOAD"
            "GDES" switches the XFOIL to geometry design stage
            "CADD" adds corner points to satisfy user specified angle threshold.
            "PANEL" regenerates vertices
            "OPER" switches the program to aerodynamic analysis mode
            "ALFA 5" angle of attack = 5
        

In [43]:
import subprocess as sp
import shutil
import sys
import string

In [50]:
ps = sp.Popen(['xfoil.exe'],
              stdin=sp.PIPE,
              stdout=None,
              stderr=None,
              encoding='utf8')

In [ ]:
def script(load_file, data_file, ALPHA, Reynold):
    load = ['load '+load_file,
            'MDES,FILT,EXEC',
            ' ',
            'PANE',
            'OPER',
            'ITER 70']
    for RE in Reynold:
        constant_set = []
              
              ['RE 750000','VISC 750000','PACC','data/S813.dat','','ALFA 0\n']

In [51]:
def issueCmd(cmd, echo=True):
    "submit a command through PIPE to the command liine"
    ps.stdin.write(cmd+'\n')
    if echo:
        print(cmd)

In [52]:
res = ps.communicate('\n'.join(['load s813.txt',
                               'MDES,FILT,EXEC',
                               ' ',
                               'PANE',
                               'OPER',
                               'ITER 70',
                               'RE 750000',
                               'VISC 750000',
                               'PACC',
                               'data/S813.dat',
                               '',
                               'ALFA 0\n']))

In [53]:
a = "123"
b = ["2"+a, "333"]

In [54]:
b

['2123', '333']